# AIS Project - Financial Transactions

- **Notebook**: Merge.ipynb
- **Contents**: This notebook contains the code to merge all data-files into 1 dataset

The merging and exporting process takes ~40 seconds, thanks to Polars being very fast :)

In [1]:
import polars as pl
import json

def merge_datasets(cards_file: str, users_file: str, transactions_file: str, train_fraud_labels:str) -> pl.DataFrame:
    """
    Merge three CSV datasets and a JSON file into a single Polars DataFrame.:
    - cards_file: Path to the cards data file, containing `client_id`.
    - users_file: Path to the users data file, containing `id` that matches `client_id` in cards_file.
    - transactions_file: Path to the transactions data file, containing `client_id`.
	- train_fraud_labels: Path to the fraud labels data file, containing `ID`.

    Returns:
        A Polars DataFrame containing the merged data.
    """

    # Read the CSV files into Polars DataFrames
    cards_data = pl.read_csv(cards_file)
    users_data = pl.read_csv(users_file)
    transactions_data = pl.read_csv(transactions_file)
    fraud_labels_df = pl.read_csv(train_fraud_labels)
    print("✅ (1/5) Reading the CSV files into Polars DataFrames...")
	
	# Ensure proper column types for merging
	## Cast all `id` columns to string to avoid mismatches
    fraud_labels_df = fraud_labels_df.with_columns(pl.col("id").cast(pl.Utf8))
    transactions_data = transactions_data.with_columns(pl.col("id").cast(pl.Utf8))
    transactions_data = transactions_data.with_columns(pl.col("client_id").cast(pl.Utf8))
    cards_data = cards_data.with_columns(pl.col("client_id").cast(pl.Utf8))
    cards_data = cards_data.with_columns(pl.col("id").cast(pl.Utf8))
    users_data = users_data.with_columns(pl.col("id").cast(pl.Utf8))

    fraud_labels_df = fraud_labels_df.rename({"id": "fraud_id"}).with_columns(pl.col("fraud_id").cast(pl.Utf8))
    transactions_data = transactions_data.rename({"id": "transaction_id"})

    print("✅ (2/5) Ensuring proper column types for merging...")

    # # Merge cards_data with users_data using `client_id` and `id`
    cards_users_merged  = cards_data.join(users_data, left_on="client_id", right_on="id", how="inner")
    print("✅ (3/5) Merging cards_data with users_data...")

	# # Merge the resulting data with transactions_data using `client_id`
    cards_users_transactions_merged = cards_users_merged.join(transactions_data, on="client_id")
    print("✅ (4/5) Merging with transactions_data...")

    # # Merge the data with fraud labels using the `id` column
    final_merged_data = cards_users_transactions_merged.join(fraud_labels_df, left_on="transaction_id", right_on="fraud_id", how="left")
    print("✅ (5/5) Merging with fraud_labels...")

    return final_merged_data

merged_data = merge_datasets(
	cards_file="./data/cards_data.csv",
	users_file="./data/users_data.csv",
	transactions_file="./data/transactions_data.csv",
	train_fraud_labels="./data/train_fraud_labels.csv"
)

✅ (1/5) Reading the CSV files into Polars DataFrames...
✅ (2/5) Ensuring proper column types for merging...
✅ (3/5) Merging cards_data with users_data...
✅ (4/5) Merging with transactions_data...
✅ (5/5) Merging with fraud_labels...


In [2]:
print(merged_data)

shape: (51_115_337, 38)
┌──────┬───────────┬────────────┬─────────────────┬───┬─────────┬──────┬────────┬──────────────────┐
│ id   ┆ client_id ┆ card_brand ┆ card_type       ┆ … ┆ zip     ┆ mcc  ┆ errors ┆ CLASS_fraud_labe │
│ ---  ┆ ---       ┆ ---        ┆ ---             ┆   ┆ ---     ┆ ---  ┆ ---    ┆ l                │
│ str  ┆ str       ┆ str        ┆ str             ┆   ┆ f64     ┆ i64  ┆ str    ┆ ---              │
│      ┆           ┆            ┆                 ┆   ┆         ┆      ┆        ┆ str              │
╞══════╪═══════════╪════════════╪═════════════════╪═══╪═════════╪══════╪════════╪══════════════════╡
│ 4333 ┆ 1556      ┆ Mastercard ┆ Debit           ┆ … ┆ 58523.0 ┆ 5499 ┆ null   ┆ No               │
│ 1955 ┆ 1556      ┆ Visa       ┆ Credit          ┆ … ┆ 58523.0 ┆ 5499 ┆ null   ┆ No               │
│ 2972 ┆ 1556      ┆ Mastercard ┆ Debit (Prepaid) ┆ … ┆ 58523.0 ┆ 5499 ┆ null   ┆ No               │
│ 412  ┆ 1556      ┆ Amex       ┆ Credit          ┆ … ┆ 58523.0 ┆ 5

In [ ]:
# merged_data.write_csv("./data/merged/merged_data.csv")
# ? This file turns out to be 12GB in size, so we actually skip this step. 
# ? Instead we export samples

In [ ]:
for n_rows in [100, 1_000, 10_000, 100_000]:
	sampled_data = merged_data.sample(n=n_rows)
	sampled_data.write_csv(f"./data/sampled_class/sampled_data_{n_rows}.csv")